# Similarity Analysis: Audio vs Lyrics Features

This notebook analyzes how similar audio and lyrics features are using multiple complementary metrics.

## Part A: Similarity Metrics (Current Analysis)
1. **Cosine Similarity** - Direct angular similarity between features
2. **Canonical Correlation Analysis (CCA)** - Find shared latent structure
3. **Cross-Modal Retrieval** - Test if one modality predicts the other

## Part B: Siamese Networks (Future Work)
- Explanation of Siamese networks for semantic similarity
- Implementation guidance
- Complexity assessment for your codebase

---
# PART A: SIMILARITY METRICS
---

In [ ]:
# Import required libraries
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.cross_decomposition import CCA
from scipy.stats import pearsonr, spearmanr
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries loaded successfully!")

## Step 1: Load Your Trained Model

We'll load your BimodalClassifier to extract audio and lyrics features.

In [ ]:
# Line 1: Import transformers for BERT
from transformers import BertModel, BertTokenizer

class VGGish_Audio_Model(nn.Module):
    """Audio feature extractor - VGGish architecture"""
    def __init__(self, num_classes=64):
        super(VGGish_Audio_Model, self).__init__()
        # Block 1: 1 channel (grayscale) → 64 feature maps
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Block 2: 64 → 128 feature maps
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Block 3: 128 → 256 feature maps
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Block 4: 256 → 512 feature maps
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        
        # Global pooling
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Classifier: 512 → 256 → 64
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Input: [batch, 1, 128, 1292] mel-spectrogram
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        
        x = self.relu(self.bn4(self.conv4(x)))
        
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        
        x = self.relu(self.fc1(x))
        x = self.fc2(x)  # Output: [batch, 64]
        
        return x

class BimodalClassifier(nn.Module):
    """Bimodal model: Audio + Lyrics → Emotion prediction"""
    def __init__(self, audio_feature_dim=64, text_feature_dim=768, num_emotions=2):
        super(BimodalClassifier, self).__init__()
        
        # Audio tower
        self.audio_model = VGGish_Audio_Model(num_classes=audio_feature_dim)
        
        # Lyrics tower (BERT frozen)
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        for param in self.bert.parameters():
            param.requires_grad = False
        
        # Fusion classifier
        combined_dim = audio_feature_dim + text_feature_dim  # 64 + 768 = 832
        self.classifier = nn.Sequential(
            nn.Linear(combined_dim, 100),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(100, num_emotions)
        )
        
    def forward(self, spectrogram, input_ids, attention_mask):
        # Line 1: Extract audio features [batch, 64]
        audio_features = self.audio_model(spectrogram)
        
        # Line 2: Extract lyrics features from BERT [CLS] token [batch, 768]
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        lyrics_features = bert_output.last_hidden_state[:, 0, :]
        
        # Line 3: Concatenate [batch, 832]
        combined_features = torch.cat((audio_features, lyrics_features), dim=1)
        
        # Line 4: Predict emotions [batch, 2] (valence, arousal)
        emotions = self.classifier(combined_features)
        
        return emotions, audio_features, lyrics_features

# Line 5: Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Line 6: Initialize and load model
# UPDATE THIS PATH to your trained model
model_path = '/content/drive/MyDrive/dissertation/bimodal_regression_model.pth'

model = BimodalClassifier(audio_feature_dim=64, text_feature_dim=768, num_emotions=2)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

print("✓ Model loaded successfully!")

## Step 2: Extract Features from Dataset

In [ ]:
# Line 1: Load your dataset CSV
# UPDATE THIS PATH to your master CSV file
csv_path = '/content/drive/MyDrive/dissertation/merge_dataset/output_from_code/master_processed_file_list.csv'
df = pd.read_csv(csv_path)

print(f"Total songs available: {len(df)}")
print(f"\nDataFrame columns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())

# Line 2: Select subset or full dataset
# For quick testing, use df.head(100)
# For full analysis, use df (all songs)
df_subset = df.head(100)  # Change this as needed
print(f"\n✓ Using {len(df_subset)} songs for analysis")

In [ ]:
def extract_features_from_dataset(model, df, device):
    """
    Extract audio and lyrics features for all songs.
    
    Returns:
        dict with:
        - audio_features: [N, 64] numpy array
        - lyrics_features: [N, 768] numpy array
        - predictions: [N, 2] numpy array (valence, arousal)
        - ground_truth: [N, 2] numpy array
        - song_ids: list of song IDs
    """
    # Line 1: Initialize storage lists
    audio_features_list = []
    lyrics_features_list = []
    predictions_list = []
    ground_truth_list = []
    song_ids = []
    
    # Line 2: Extract features without computing gradients
    model.eval()
    with torch.no_grad():
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting features"):
            try:
                # Line 3: Load spectrogram [128, 1292]
                spectrogram = np.load(row['spectrogram_path'])
                # Line 4: Convert to tensor [1, 1, 128, 1292]
                spectrogram = torch.FloatTensor(spectrogram).unsqueeze(0).unsqueeze(0).to(device)
                
                # Line 5: Load tokenized lyrics
                lyrics_tokens = torch.load(row['lyrics_path'])
                input_ids = lyrics_tokens['input_ids'].to(device)
                attention_mask = lyrics_tokens['attention_mask'].to(device)
                
                # Line 6: Forward pass
                preds, audio_feat, lyrics_feat = model(spectrogram, input_ids, attention_mask)
                
                # Line 7: Store results (move to CPU and convert to numpy)
                audio_features_list.append(audio_feat.squeeze().cpu().numpy())
                lyrics_features_list.append(lyrics_feat.squeeze().cpu().numpy())
                predictions_list.append(preds.squeeze().cpu().numpy())
                ground_truth_list.append([row['valence'], row['arousal']])
                song_ids.append(row['song_id'])
                
            except Exception as e:
                print(f"Error processing {row['song_id']}: {e}")
                continue
    
    # Line 8: Convert lists to arrays
    audio_features = np.stack(audio_features_list)      # [N, 64]
    lyrics_features = np.stack(lyrics_features_list)    # [N, 768]
    predictions = np.stack(predictions_list)            # [N, 2]
    ground_truth = np.stack(ground_truth_list)          # [N, 2]
    
    print(f"\n✓ Extracted features for {len(song_ids)} songs")
    print(f"  Audio features shape: {audio_features.shape}")
    print(f"  Lyrics features shape: {lyrics_features.shape}")
    
    return {
        'audio_features': audio_features,
        'lyrics_features': lyrics_features,
        'predictions': predictions,
        'ground_truth': ground_truth,
        'song_ids': song_ids
    }

# Extract features
features_dict = extract_features_from_dataset(model, df_subset, device)

---
## Method 1: Cosine Similarity

**What it measures**: Angular similarity between feature vectors (ranges from -1 to 1)

**Key insight**: Diagonal of cross-modal matrix = similarity between each song's audio and its OWN lyrics

**Interpretation**:
- High diagonal values (>0.7): Audio and lyrics are strongly aligned
- Moderate (0.5-0.7): Some alignment
- Low (<0.5): Audio and lyrics encode different aspects

In [ ]:
def compute_cosine_similarity_analysis(audio_features, lyrics_features):
    """
    Compute all pairwise cosine similarities.
    """
    # Line 1: Audio-to-audio similarity [N, N]
    # Entry [i,j] = similarity between audio_i and audio_j
    audio_sim = cosine_similarity(audio_features, audio_features)
    
    # Line 2: Lyrics-to-lyrics similarity [N, N]
    lyrics_sim = cosine_similarity(lyrics_features, lyrics_features)
    
    # Line 3: CROSS-MODAL similarity [N, N]
    # Entry [i,j] = similarity between audio_i and lyrics_j
    # KEY METRIC: Diagonal elements = audio_i vs lyrics_i (same song)
    cross_modal_sim = cosine_similarity(audio_features, lyrics_features)
    
    # Line 4: Extract diagonal (self-similarity)
    self_similarity = np.diag(cross_modal_sim)
    
    # Line 5: Extract off-diagonal (cross-song similarity)
    mask = np.ones_like(cross_modal_sim, dtype=bool)
    np.fill_diagonal(mask, False)
    cross_song_sim = cross_modal_sim[mask]
    
    # Line 6: Print results
    print("="*70)
    print("COSINE SIMILARITY ANALYSIS")
    print("="*70)
    print(f"\n1. SELF-SIMILARITY (audio vs own lyrics):")
    print(f"   Mean:  {self_similarity.mean():.4f}")
    print(f"   Std:   {self_similarity.std():.4f}")
    print(f"   Range: [{self_similarity.min():.4f}, {self_similarity.max():.4f}]")
    
    print(f"\n2. CROSS-SONG SIMILARITY (audio_i vs lyrics_j, i≠j):")
    print(f"   Mean:  {cross_song_sim.mean():.4f}")
    print(f"   Std:   {cross_song_sim.std():.4f}")
    
    print(f"\n3. WITHIN-MODALITY SIMILARITY:")
    print(f"   Audio-to-audio mean:   {audio_sim[mask].mean():.4f}")
    print(f"   Lyrics-to-lyrics mean: {lyrics_sim[mask].mean():.4f}")
    
    # Line 7: Interpretation
    if self_similarity.mean() > 0.7:
        print(f"\n✓ STRONG alignment: Audio and lyrics are highly similar")
    elif self_similarity.mean() > 0.5:
        print(f"\n✓ MODERATE alignment: Some similarity between audio and lyrics")
    else:
        print(f"\n! WEAK alignment: Audio and lyrics encode different information")
    
    return audio_sim, lyrics_sim, cross_modal_sim, self_similarity

# Run analysis
audio_sim, lyrics_sim, cross_modal_sim, self_sim = compute_cosine_similarity_analysis(
    features_dict['audio_features'],
    features_dict['lyrics_features']
)

In [ ]:
# Visualize similarity matrices
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Line 1: Plot audio similarity
im1 = axes[0].imshow(audio_sim, cmap='coolwarm', vmin=0, vmax=1, aspect='auto')
axes[0].set_title('Audio-to-Audio Similarity', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Song Index')
axes[0].set_ylabel('Song Index')
plt.colorbar(im1, ax=axes[0], fraction=0.046)

# Line 2: Plot lyrics similarity
im2 = axes[1].imshow(lyrics_sim, cmap='coolwarm', vmin=0, vmax=1, aspect='auto')
axes[1].set_title('Lyrics-to-Lyrics Similarity', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Song Index')
axes[1].set_ylabel('Song Index')
plt.colorbar(im2, ax=axes[1], fraction=0.046)

# Line 3: Plot cross-modal similarity (KEY PLOT)
im3 = axes[2].imshow(cross_modal_sim, cmap='coolwarm', vmin=0, vmax=1, aspect='auto')
axes[2].set_title('Audio-to-Lyrics Cross-Modal Similarity', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Lyrics Index')
axes[2].set_ylabel('Audio Index')
plt.colorbar(im3, ax=axes[2], fraction=0.046)

plt.tight_layout()
plt.show()

# Line 4: Plot histogram of self-similarity
plt.figure(figsize=(10, 6))
plt.hist(self_sim, bins=30, color='steelblue', alpha=0.7, edgecolor='black')
plt.axvline(self_sim.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {self_sim.mean():.3f}')
plt.xlabel('Cosine Similarity (audio vs own lyrics)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Self-Similarity Scores', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.show()

---
## Method 2: Canonical Correlation Analysis (CCA)

**What it does**: Finds linear transformations that maximize correlation between audio and lyrics

**Key insight**: Discovers if there are shared latent dimensions (e.g., "energy" encoded in both modalities)

**Interpretation**:
- High canonical correlations (>0.7): Strong shared structure
- Moderate (0.5-0.7): Some shared dimensions
- Low (<0.5): Modalities are complementary, not redundant

In [ ]:
def perform_cca_analysis(audio_features, lyrics_features, n_components=10):
    """
    Canonical Correlation Analysis between audio and lyrics.
    """
    # Line 1: Initialize CCA
    # n_components: how many shared dimensions to find
    cca = CCA(n_components=n_components, max_iter=1000)
    
    # Line 2: Fit CCA to learn transformations
    # Finds W_audio and W_lyrics that maximize correlation
    cca.fit(audio_features, lyrics_features)
    
    # Line 3: Transform features to canonical space
    audio_canonical, lyrics_canonical = cca.transform(audio_features, lyrics_features)
    
    # Line 4: Compute correlation for each canonical component
    correlations = []
    for i in range(n_components):
        # Pearson correlation between i-th canonical dimension
        corr, p_value = pearsonr(audio_canonical[:, i], lyrics_canonical[:, i])
        correlations.append(corr)
    
    correlations = np.array(correlations)
    
    # Line 5: Print results
    print("="*70)
    print("CANONICAL CORRELATION ANALYSIS (CCA)")
    print("="*70)
    print(f"\nCanonical correlations (n={n_components}):")
    for i, corr in enumerate(correlations):
        print(f"  Component {i+1}: {corr:.4f}")
    
    print(f"\nSummary statistics:")
    print(f"  Mean correlation: {correlations.mean():.4f}")
    print(f"  Max correlation:  {correlations.max():.4f}")
    print(f"  Std:              {correlations.std():.4f}")
    
    # Line 6: Interpretation
    if correlations[0] > 0.7:
        print(f"\n✓ STRONG shared structure: First component correlation = {correlations[0]:.3f}")
    elif correlations[0] > 0.5:
        print(f"\n✓ MODERATE shared structure: Some shared latent dimensions")
    else:
        print(f"\n! LIMITED shared structure: Modalities may be complementary")
    
    return cca, correlations, audio_canonical, lyrics_canonical

# Run CCA
cca_model, cca_corrs, audio_can, lyrics_can = perform_cca_analysis(
    features_dict['audio_features'],
    features_dict['lyrics_features'],
    n_components=10
)

In [ ]:
# Visualize CCA results
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Line 1: Bar chart of canonical correlations
axes[0].bar(range(1, len(cca_corrs) + 1), cca_corrs, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axhline(y=0.5, color='red', linestyle='--', linewidth=2, label='Moderate threshold (0.5)')
axes[0].axhline(y=0.7, color='darkred', linestyle='--', linewidth=2, label='Strong threshold (0.7)')
axes[0].set_xlabel('Canonical Component', fontsize=12)
axes[0].set_ylabel('Correlation Coefficient', fontsize=12)
axes[0].set_title('Canonical Correlations', fontsize=14, fontweight='bold')
axes[0].set_ylim([0, 1])
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Line 2: Scatter plot of first canonical component
axes[1].scatter(audio_can[:, 0], lyrics_can[:, 0], alpha=0.6, s=50, color='purple', edgecolors='black', linewidth=0.5)
axes[1].set_xlabel('Audio Canonical Component 1', fontsize=12)
axes[1].set_ylabel('Lyrics Canonical Component 1', fontsize=12)
axes[1].set_title(f'First Canonical Component (r={cca_corrs[0]:.3f})', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Add correlation line
z = np.polyfit(audio_can[:, 0], lyrics_can[:, 0], 1)
p = np.poly1d(z)
axes[1].plot(audio_can[:, 0], p(audio_can[:, 0]), "r--", linewidth=2, label='Linear fit')
axes[1].legend()

plt.tight_layout()
plt.show()

---
## Method 3: Cross-Modal Retrieval

**What it does**: Tests if audio features can retrieve matching lyrics (and vice versa)

**Key insight**: Measures how well one modality predicts the other

**Interpretation**:
- High Top-5 accuracy (>50%): Strong predictive relationship
- Moderate (20-50%): Some predictability
- Low (<20%): One modality doesn't predict the other well

In [ ]:
def cross_modal_retrieval_analysis(audio_features, lyrics_features, top_k=5):
    """
    Perform cross-modal retrieval task.
    """
    # Line 1: Compute cross-modal similarity matrix
    sim_matrix = cosine_similarity(audio_features, lyrics_features)
    n_samples = len(audio_features)
    
    # Line 2: Audio → Lyrics retrieval
    # For each audio, find top-k most similar lyrics
    audio_to_lyrics_top_k = np.argsort(sim_matrix, axis=1)[:, ::-1][:, :top_k]
    
    # Line 3: Check if correct match is in top-k
    audio_to_lyrics_hits = []
    for i in range(n_samples):
        if i in audio_to_lyrics_top_k[i]:
            audio_to_lyrics_hits.append(1)
        else:
            audio_to_lyrics_hits.append(0)
    
    audio_to_lyrics_acc = np.mean(audio_to_lyrics_hits)
    
    # Line 4: Lyrics → Audio retrieval
    lyrics_to_audio_top_k = np.argsort(sim_matrix.T, axis=1)[:, ::-1][:, :top_k]
    
    lyrics_to_audio_hits = []
    for i in range(n_samples):
        if i in lyrics_to_audio_top_k[i]:
            lyrics_to_audio_hits.append(1)
        else:
            lyrics_to_audio_hits.append(0)
    
    lyrics_to_audio_acc = np.mean(lyrics_to_audio_hits)
    
    # Line 5: Also compute Top-1 and Top-10 accuracy
    # Top-1 (exact match)
    audio_to_lyrics_top1 = np.argmax(sim_matrix, axis=1)
    top1_acc = np.mean(audio_to_lyrics_top1 == np.arange(n_samples))
    
    # Top-10
    if n_samples >= 10:
        audio_to_lyrics_top_10 = np.argsort(sim_matrix, axis=1)[:, ::-1][:, :10]
        top10_hits = [i in audio_to_lyrics_top_10[i] for i in range(n_samples)]
        top10_acc = np.mean(top10_hits)
    else:
        top10_acc = None
    
    # Line 6: Print results
    print("="*70)
    print("CROSS-MODAL RETRIEVAL ANALYSIS")
    print("="*70)
    print(f"\n1. RETRIEVAL ACCURACY:")
    print(f"   Audio → Lyrics (Top-{top_k}): {audio_to_lyrics_acc:.2%}")
    print(f"   Lyrics → Audio (Top-{top_k}): {lyrics_to_audio_acc:.2%}")
    
    print(f"\n2. ADDITIONAL METRICS:")
    print(f"   Top-1 accuracy (exact match):  {top1_acc:.2%}")
    if top10_acc:
        print(f"   Top-10 accuracy:               {top10_acc:.2%}")
    
    # Line 7: Interpretation
    print(f"\n3. INTERPRETATION:")
    if audio_to_lyrics_acc > 0.5:
        print(f"   ✓ GOOD alignment: Audio features predict matching lyrics well")
    elif audio_to_lyrics_acc > 0.2:
        print(f"   ✓ MODERATE alignment: Some predictive power")
    else:
        print(f"   ! WEAK alignment: Limited cross-modal predictability")
    
    print(f"\n   Meaning: {audio_to_lyrics_acc:.1%} of the time, given a song's audio,")
    print(f"   the correct lyrics are in the top-{top_k} most similar lyrics.")
    
    return {
        'audio_to_lyrics_acc': audio_to_lyrics_acc,
        'lyrics_to_audio_acc': lyrics_to_audio_acc,
        'top1_acc': top1_acc,
        'top10_acc': top10_acc
    }

# Run retrieval analysis
retrieval_results = cross_modal_retrieval_analysis(
    features_dict['audio_features'],
    features_dict['lyrics_features'],
    top_k=5
)

In [ ]:
# Visualize retrieval accuracy at different k values
k_values = [1, 2, 3, 5, 10, 20]
accuracies = []

sim_matrix = cosine_similarity(features_dict['audio_features'], features_dict['lyrics_features'])
n_samples = len(features_dict['audio_features'])

# Line 1: Compute accuracy for different k values
for k in k_values:
    if k <= n_samples:
        top_k_indices = np.argsort(sim_matrix, axis=1)[:, ::-1][:, :k]
        hits = [i in top_k_indices[i] for i in range(n_samples)]
        accuracies.append(np.mean(hits))
    else:
        accuracies.append(None)

# Line 2: Plot
plt.figure(figsize=(10, 6))
valid_k = [k for k, acc in zip(k_values, accuracies) if acc is not None]
valid_acc = [acc for acc in accuracies if acc is not None]

plt.plot(valid_k, valid_acc, marker='o', linewidth=2, markersize=8, color='steelblue')
plt.xlabel('Top-K', fontsize=12)
plt.ylabel('Retrieval Accuracy', fontsize=12)
plt.title('Cross-Modal Retrieval Accuracy (Audio → Lyrics)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.ylim([0, 1])

# Add value labels
for k, acc in zip(valid_k, valid_acc):
    plt.text(k, acc + 0.03, f'{acc:.2%}', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

---
## Summary Report

Comprehensive summary of all three similarity analyses.

In [ ]:
def generate_comprehensive_report(features_dict, self_sim, cca_corrs, retrieval_results):
    """
    Generate final summary report.
    """
    print("\n\n")
    print("="*80)
    print(" "*25 + "COMPREHENSIVE SIMILARITY REPORT")
    print("="*80)
    
    print(f"\n📊 DATASET SUMMARY")
    print(f"{'-'*80}")
    print(f"   Total songs analyzed: {len(features_dict['song_ids'])}")
    print(f"   Audio feature dim:    {features_dict['audio_features'].shape[1]}")
    print(f"   Lyrics feature dim:   {features_dict['lyrics_features'].shape[1]}")
    
    print(f"\n📏 METHOD 1: COSINE SIMILARITY")
    print(f"{'-'*80}")
    print(f"   Self-similarity (audio vs own lyrics):")
    print(f"     Mean: {self_sim.mean():.4f}")
    print(f"     Std:  {self_sim.std():.4f}")
    
    if self_sim.mean() > 0.7:
        print(f"   ✓ STRONG alignment between audio and lyrics")
    elif self_sim.mean() > 0.5:
        print(f"   ✓ MODERATE alignment")
    else:
        print(f"   ! WEAK alignment - modalities encode different aspects")
    
    print(f"\n🔗 METHOD 2: CANONICAL CORRELATION ANALYSIS")
    print(f"{'-'*80}")
    print(f"   Top 3 canonical correlations:")
    for i in range(min(3, len(cca_corrs))):
        print(f"     Component {i+1}: {cca_corrs[i]:.4f}")
    
    if cca_corrs[0] > 0.7:
        print(f"   ✓ STRONG shared latent structure")
    elif cca_corrs[0] > 0.5:
        print(f"   ✓ MODERATE shared structure")
    else:
        print(f"   ! LIMITED shared structure - complementary modalities")
    
    print(f"\n🔍 METHOD 3: CROSS-MODAL RETRIEVAL")
    print(f"{'-'*80}")
    print(f"   Audio → Lyrics (Top-5): {retrieval_results['audio_to_lyrics_acc']:.2%}")
    print(f"   Lyrics → Audio (Top-5): {retrieval_results['lyrics_to_audio_acc']:.2%}")
    print(f"   Top-1 exact match:      {retrieval_results['top1_acc']:.2%}")
    
    if retrieval_results['audio_to_lyrics_acc'] > 0.5:
        print(f"   ✓ GOOD cross-modal predictability")
    elif retrieval_results['audio_to_lyrics_acc'] > 0.2:
        print(f"   ✓ MODERATE predictability")
    else:
        print(f"   ! WEAK predictability")
    
    print(f"\n💡 OVERALL CONCLUSION")
    print(f"{'-'*80}")
    
    # Overall assessment
    scores = [
        self_sim.mean(),
        cca_corrs[0],
        retrieval_results['audio_to_lyrics_acc']
    ]
    avg_score = np.mean(scores)
    
    if avg_score > 0.6:
        print(f"   The audio and lyrics features show STRONG similarity and alignment.")
        print(f"   They encode related semantic information and have predictive power.")
    elif avg_score > 0.4:
        print(f"   The audio and lyrics features show MODERATE similarity.")
        print(f"   They share some structure but also encode unique information.")
    else:
        print(f"   The audio and lyrics features show LIMITED similarity.")
        print(f"   They appear to encode COMPLEMENTARY rather than redundant information.")
        print(f"   This suggests both modalities contribute unique value to emotion prediction.")
    
    print("\n" + "="*80)

# Generate report
generate_comprehensive_report(features_dict, self_sim, cca_corrs, retrieval_results)

---
## Save Results

In [ ]:
# Save all results
import os

# Line 1: Create output directory
output_dir = '/content/drive/MyDrive/dissertation/similarity_analysis_results/'
os.makedirs(output_dir, exist_ok=True)

# Line 2: Save similarity matrices
np.save(os.path.join(output_dir, 'audio_similarity_matrix.npy'), audio_sim)
np.save(os.path.join(output_dir, 'lyrics_similarity_matrix.npy'), lyrics_sim)
np.save(os.path.join(output_dir, 'cross_modal_similarity_matrix.npy'), cross_modal_sim)

# Line 3: Save CCA results
np.save(os.path.join(output_dir, 'cca_correlations.npy'), cca_corrs)
np.save(os.path.join(output_dir, 'audio_canonical.npy'), audio_can)
np.save(os.path.join(output_dir, 'lyrics_canonical.npy'), lyrics_can)

# Line 4: Save extracted features
np.save(os.path.join(output_dir, 'audio_features.npy'), features_dict['audio_features'])
np.save(os.path.join(output_dir, 'lyrics_features.npy'), features_dict['lyrics_features'])

# Line 5: Create summary CSV
results_df = pd.DataFrame({
    'song_id': features_dict['song_ids'],
    'self_similarity': self_sim,
    'true_valence': features_dict['ground_truth'][:, 0],
    'true_arousal': features_dict['ground_truth'][:, 1],
    'pred_valence': features_dict['predictions'][:, 0],
    'pred_arousal': features_dict['predictions'][:, 1]
})
results_df.to_csv(os.path.join(output_dir, 'similarity_summary.csv'), index=False)

# Line 6: Save metrics summary
metrics = {
    'mean_self_similarity': float(self_sim.mean()),
    'std_self_similarity': float(self_sim.std()),
    'cca_correlation_1': float(cca_corrs[0]),
    'cca_mean_correlation': float(cca_corrs.mean()),
    'retrieval_audio_to_lyrics': float(retrieval_results['audio_to_lyrics_acc']),
    'retrieval_lyrics_to_audio': float(retrieval_results['lyrics_to_audio_acc']),
    'retrieval_top1': float(retrieval_results['top1_acc'])
}

import json
with open(os.path.join(output_dir, 'metrics_summary.json'), 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"✓ Results saved to: {output_dir}")
print(f"\nFiles created:")
print(f"  - audio_similarity_matrix.npy")
print(f"  - lyrics_similarity_matrix.npy")
print(f"  - cross_modal_similarity_matrix.npy")
print(f"  - cca_correlations.npy")
print(f"  - audio_canonical.npy")
print(f"  - lyrics_canonical.npy")
print(f"  - audio_features.npy")
print(f"  - lyrics_features.npy")
print(f"  - similarity_summary.csv")
print(f"  - metrics_summary.json")

---
---
# PART B: SIAMESE NETWORKS
---
---

## What is a Siamese Network?

A **Siamese network** is a neural network architecture designed to learn semantic similarity between inputs. It consists of:

1. **Two identical subnetworks** (shared weights) that process each modality
2. **A similarity metric** (e.g., distance or dot product) between the outputs
3. **Contrastive or triplet loss** that trains the network to:
   - Pull together matching pairs (audio_i, lyrics_i)
   - Push apart non-matching pairs (audio_i, lyrics_j where i≠j)

### Architecture Diagram:

```
Audio Input                    Lyrics Input
     ↓                              ↓
[Spectrogram]                [Tokenized Text]
     ↓                              ↓
[Audio Encoder]              [Lyrics Encoder]
(VGGish/CNN)                 (BERT/Transformer)
     ↓                              ↓
[Audio Embedding]            [Lyrics Embedding]
(Shared dim: e.g., 256)      (Shared dim: e.g., 256)
     ↓                              ↓
     └──────── Distance/Similarity ────────┘
                    ↓
              [Loss Function]
          (Contrastive or Triplet)
```

---

## A) What Will a Siamese Network Tell You?

### Compared to your current similarity analysis:

| Method | What it tells you |
|--------|------------------|
| **Cosine Similarity** | Uses features from models trained for **emotion prediction** | 
| **CCA** | Finds linear correlations in **existing** feature spaces |
| **Siamese Network** | Learns a **new embedding space** explicitly optimized for cross-modal similarity |

### Specific insights from Siamese networks:

1. **Learned semantic similarity**: Instead of using emotion-prediction features, the network learns what "similar" means for audio-lyrics pairs

2. **Improved retrieval**: After training, audio→lyrics retrieval should be much better because the model is explicitly trained for this task

3. **Embedding quality**: You can visualize the learned embedding space (using t-SNE or UMAP) to see if matching audio-lyrics pairs cluster together

4. **Transferability**: The learned embeddings can be used for:
   - Song recommendation (find songs with similar audio-lyrics characteristics)
   - Anomaly detection (find songs where audio and lyrics don't match)
   - Zero-shot learning (predict properties of new songs)

5. **What's different from current approach**:
   - **Current**: Your features are optimized for valence/arousal prediction
   - **Siamese**: Features are optimized for audio-lyrics similarity
   - These may capture different aspects of the songs!

---

## B) How to Implement a Siamese Network

### Implementation Steps:

#### Step 1: Design the Architecture

You need:
1. **Audio encoder**: Your VGGish model (reuse existing architecture)
2. **Lyrics encoder**: Your BERT model (reuse existing architecture)
3. **Projection heads**: Map both to same dimension (e.g., 256)
4. **Loss function**: Contrastive loss or triplet loss

```python
class SiameseNetwork(nn.Module):
    def __init__(self, audio_encoder, lyrics_encoder, embedding_dim=256):
        super().__init__()
        
        # Line 1: Reuse your existing encoders
        self.audio_encoder = audio_encoder  # VGGish → [batch, 64]
        self.lyrics_encoder = lyrics_encoder  # BERT → [batch, 768]
        
        # Line 2: Projection heads to shared embedding space
        self.audio_projection = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, embedding_dim),
            nn.LayerNorm(embedding_dim)
        )
        
        self.lyrics_projection = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Linear(512, embedding_dim),
            nn.LayerNorm(embedding_dim)
        )
    
    def forward(self, audio, lyrics_ids, lyrics_mask):
        # Line 3: Extract features
        audio_feat = self.audio_encoder(audio)  # [B, 64]
        lyrics_feat = self.lyrics_encoder(lyrics_ids, lyrics_mask)  # [B, 768]
        
        # Line 4: Project to shared embedding space
        audio_emb = self.audio_projection(audio_feat)  # [B, 256]
        lyrics_emb = self.lyrics_projection(lyrics_feat)  # [B, 256]
        
        # Line 5: L2 normalize for cosine similarity
        audio_emb = F.normalize(audio_emb, p=2, dim=1)
        lyrics_emb = F.normalize(lyrics_emb, p=2, dim=1)
        
        return audio_emb, lyrics_emb
```

#### Step 2: Choose a Loss Function

**Option 1: Contrastive Loss (Simpler)**

```python
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin
    
    def forward(self, audio_emb, lyrics_emb, labels):
        # Line 1: Compute pairwise distances
        # labels: 1 if matching pair, 0 if not
        distances = F.pairwise_distance(audio_emb, lyrics_emb)
        
        # Line 2: Contrastive loss
        # For matching pairs: minimize distance
        # For non-matching pairs: maximize distance (up to margin)
        loss = labels * distances.pow(2) + \
               (1 - labels) * F.relu(self.margin - distances).pow(2)
        
        return loss.mean()
```

**Option 2: Triplet Loss (Better performance)**

```python
class TripletLoss(nn.Module):
    def __init__(self, margin=0.2):
        super().__init__()
        self.margin = margin
    
    def forward(self, anchor, positive, negative):
        # Line 1: Compute distances
        # anchor: audio_i
        # positive: lyrics_i (matching)
        # negative: lyrics_j (non-matching)
        pos_dist = F.pairwise_distance(anchor, positive)
        neg_dist = F.pairwise_distance(anchor, negative)
        
        # Line 2: Triplet loss
        # Want: distance(anchor, positive) + margin < distance(anchor, negative)
        loss = F.relu(pos_dist - neg_dist + self.margin)
        
        return loss.mean()
```

**Option 3: InfoNCE Loss (State-of-the-art, used in CLIP)**

```python
class InfoNCELoss(nn.Module):
    def __init__(self, temperature=0.07):
        super().__init__()
        self.temperature = temperature
    
    def forward(self, audio_emb, lyrics_emb):
        # Line 1: Compute similarity matrix
        # [batch_size, batch_size]
        logits = torch.matmul(audio_emb, lyrics_emb.T) / self.temperature
        
        # Line 2: Create labels (diagonal = positive pairs)
        batch_size = audio_emb.size(0)
        labels = torch.arange(batch_size, device=audio_emb.device)
        
        # Line 3: Cross-entropy loss (both directions)
        loss_audio = F.cross_entropy(logits, labels)
        loss_lyrics = F.cross_entropy(logits.T, labels)
        
        return (loss_audio + loss_lyrics) / 2
```

**RECOMMENDATION**: Use **InfoNCE loss** (Option 3) - it's the most effective and easiest to implement.

#### Step 3: Training Loop

```python
# Line 1: Initialize model
siamese_model = SiameseNetwork(audio_encoder, lyrics_encoder, embedding_dim=256)
criterion = InfoNCELoss(temperature=0.07)
optimizer = torch.optim.Adam(siamese_model.parameters(), lr=1e-4)

# Line 2: Training loop
for epoch in range(num_epochs):
    for batch in dataloader:
        audio, lyrics_ids, lyrics_mask = batch
        
        # Line 3: Forward pass
        audio_emb, lyrics_emb = siamese_model(audio, lyrics_ids, lyrics_mask)
        
        # Line 4: Compute loss
        loss = criterion(audio_emb, lyrics_emb)
        
        # Line 5: Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
```

#### Step 4: Evaluation

```python
# Line 1: Extract embeddings for test set
siamese_model.eval()
with torch.no_grad():
    audio_embeddings = []
    lyrics_embeddings = []
    
    for batch in test_loader:
        audio_emb, lyrics_emb = siamese_model(audio, lyrics_ids, lyrics_mask)
        audio_embeddings.append(audio_emb.cpu())
        lyrics_embeddings.append(lyrics_emb.cpu())

# Line 2: Compute retrieval accuracy
similarity_matrix = cosine_similarity(audio_embeddings, lyrics_embeddings)
top_k_acc = compute_retrieval_accuracy(similarity_matrix, k=5)

print(f"Retrieval accuracy: {top_k_acc:.2%}")
```

---

## C) Implementation Complexity Assessment

### Effort Estimation: **MODERATE** (2-3 days of work)

#### What you can REUSE from your current code:

✅ **Easy to reuse**:
1. **Data loading**: Your existing dataloader (spectrograms + lyrics)
2. **Audio encoder**: Your VGGish_Audio_Model architecture
3. **Lyrics encoder**: Your BERT model
4. **Preprocessing**: All your spectrogram and tokenization code
5. **Training infrastructure**: Device management, saving/loading models

#### What you need to ADD:

📝 **New code required** (~300-400 lines):
1. **SiameseNetwork class**: ~50 lines
   - Wrapper around your existing encoders
   - Add projection heads (2 small MLPs)

2. **Loss function**: ~20 lines
   - InfoNCE loss is simplest and best

3. **Modified training loop**: ~100 lines
   - Different loss computation
   - No emotion prediction head needed

4. **Evaluation code**: ~50 lines
   - Retrieval accuracy computation
   - Embedding visualization (t-SNE/UMAP)

5. **Data augmentation** (optional but recommended): ~50 lines
   - Audio: time masking, frequency masking
   - Lyrics: random word masking

#### Detailed Complexity Breakdown:

| Task | Difficulty | Time | Notes |
|------|-----------|------|-------|
| **1. Architecture setup** | Easy | 2-3 hours | Mostly copy-paste from current model |
| **2. Loss function** | Easy | 1 hour | Use InfoNCE (simple implementation) |
| **3. Modify dataloader** | Easy | 1-2 hours | Ensure batches have positive pairs |
| **4. Training loop** | Medium | 3-4 hours | Different from classification loop |
| **5. Evaluation** | Medium | 2-3 hours | Retrieval metrics + visualization |
| **6. Debugging** | Medium | 3-5 hours | Checking convergence, tuning hyperparams |
| **7. Experiments** | Low | 4-6 hours | Training on full dataset |
| **Total** | **MODERATE** | **16-24 hours** | ~2-3 days of focused work |

#### Key Decisions:

1. **Freeze encoders or fine-tune?**
   - **Freeze** (easier, faster): Use pretrained features, only train projection heads
   - **Fine-tune** (better, slower): Update entire network
   - **Recommendation**: Start frozen, then try fine-tuning

2. **Embedding dimension?**
   - Typical: 128, 256, or 512
   - **Recommendation**: 256 (good balance)

3. **Batch size?**
   - Larger is better for contrastive learning (more negative pairs)
   - **Recommendation**: 64 or 128 if GPU memory allows

4. **Temperature (for InfoNCE)?**
   - Range: 0.05 - 0.1
   - **Recommendation**: 0.07 (from CLIP paper)

---

## Complete Implementation Code Structure

Here's the full file structure you'd need:

```
siamese_network/
├── model.py              # SiameseNetwork class (~50 lines)
├── loss.py               # InfoNCE loss (~20 lines)
├── train.py              # Training loop (~100 lines)
├── evaluate.py           # Evaluation metrics (~50 lines)
├── dataset.py            # REUSE your existing dataloader
└── main.py               # Entry point (~30 lines)
```

**Total new code**: ~250 lines (not counting reused components)

---

## Comparison: Current Approach vs Siamese Network

| Aspect | Current (Similarity Metrics) | Siamese Network |
|--------|------------------------------|------------------|
| **Purpose** | Analyze existing features | Learn similarity-optimized features |
| **Training** | No additional training needed | Requires training (~4-6 hours) |
| **Retrieval accuracy** | Baseline (e.g., 20-40%) | Improved (e.g., 60-80%) |
| **Interpretability** | High (using known features) | Medium (learned representations) |
| **Computation** | Fast (just inference) | Moderate (needs training) |
| **For dissertation** | Good for analysis | Good for novel contribution |

---

## Recommendation

### Do BOTH:

1. **First**: Run the similarity analysis (Part A of this notebook)
   - Quick to execute
   - Establishes baseline
   - Good for dissertation analysis section

2. **Then**: Implement Siamese network
   - ~2-3 days of work
   - Shows improved performance
   - Strong contribution for dissertation
   - Demonstrates ability to design novel architectures

### For your dissertation:

**Chapter structure**:
- **Section 1**: "Similarity Analysis of Learned Features" (Part A)
- **Section 2**: "Learning Cross-Modal Embeddings with Siamese Networks" (Part B)
- **Section 3**: "Comparative Analysis" (Which approach works better?)

This gives you a strong narrative: analyze existing features → design better approach → demonstrate improvement.

---

## Next Steps

1. ✅ Run Part A (similarity analysis) first
2. Review results and interpret findings
3. If interested in Siamese networks, I can provide:
   - Complete implementation code
   - Training script adapted to your data
   - Evaluation code for retrieval metrics

Would you like me to implement the Siamese network code?